In [1]:
import cv2 
import os
import uuid
import IPython
import ipywidgets as widgets
from IPython.display import display
from ratelimiter import RateLimiter

ipython = IPython.get_ipython()

SESSION = str(uuid.uuid1())
IDX = 1
DATA = "data"
FPS = 10
THEFT = -1

def on_button_clicked(b):
    global THEFT, IDX
    THEFT = IDX
    print("Theft IDX: %d" % THEFT)
    
w_image = widgets.Image(format='png',
                        layout=widgets.Layout(width='100%'))

w_button = widgets.Button(description="Theft")
w_button.on_click(on_button_clicked)

w_vbox = widgets.VBox([w_button, w_image])

display(w_vbox)

cap = cv2.VideoCapture(0)

rate_limiter = RateLimiter(max_calls=1, period=1/FPS)

if not os.path.exists('sessions.csv'):
    open('sessions.csv', 'w').write('uuid,fps,frames,theft_idx\n')

try:    
    while True:
        with rate_limiter:
            # Have to call this to get update values from sliders / dropdowns
            ipython.kernel.do_one_iteration()

            # Read the frame from the camera
            ret, frame = cap.read()

            result, img_png = cv2.imencode('.png', frame)
            w_image.value = img_png.tobytes()
            
            open(os.path.join(DATA, "%s_%d.png" % (SESSION, IDX)), 'wb').write(img_png)
            
            IDX += 1

except KeyboardInterrupt:
    pass
finally:
    cap.release()
    open('sessions.csv', 'a+').write("%s,%d,%d,%d\n" % (SESSION, FPS, IDX, THEFT))



Theft IDX: 6
